# Benchmark of existing approaches for detecting machine-generated text

## Contents 
1. [Solaiman](#Solaiman)
    1. [Install dependencies](#Install-dependencies)
    1. [Solaiman Code](#Solaiman-Code)

## Solaiman

_Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford,
Gretchen Krueger, Jong Wook Kim, Sarah Kreps, Miles McCain, Alex Newhouse, Jason Blazakis, Kris McGuffie, and Jasmine Wang. 2019. Release strategies and the social impacts of language models._

### Install dependencies
Dependencies from: [https://github.com/HendrikStrobelt/detecting-fake-text/blob/master/requirements.txt](detecting-fake-text/requirements.txt).

### Solaiman Code
Source code: [https://github.com/openai/gpt-2-output-dataset/tree/master/detector](https://github.com/openai/gpt-2-output-dataset/tree/master/detector)

Logic extracted form `server.py`

In [4]:
import transformers
assert transformers.__version__ == '2.9.1', "Use transformers 2.9.1. It is available in the conda environment transf291"

In [6]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import json
import torch
from urllib.parse import urlparse, unquote

#### Select model to use RoBERTa base or large

In [26]:
# model_name = 'roberta-large'
model_name = 'roberta-base'

#### Define model, tokenizer, and basic functions

In [27]:
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
device='cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
def evaluate(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.max_len - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()

    return json.dumps(dict(
        all_tokens=all_tokens,
        used_tokens=used_tokens,
        real_probability=real,
        fake_probability=fake
    ))


def initialize(checkpoint):
#     if checkpoint.startswith('gs://'):
#         print(f'Downloading {checkpoint}', file=sys.stderr)
#         subprocess.check_output(['gsutil', 'cp', checkpoint, '.'])
#         checkpoint = os.path.basename(checkpoint)
#         assert os.path.isfile(checkpoint)

    print(f'Loading checkpoint from {checkpoint}')
    data = torch.load(checkpoint, map_location='cpu')
    model.load_state_dict(data['model_state_dict'])
    model.eval()

#### Download finetuned model and load it into RoBERTa

In [24]:
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-base.pt
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-large.pt

In [17]:
# initialize('detector-large.pt')
initialize('detector-base.pt')

Loading checkpoint from detector-base.pt


In [25]:
evaluate("hello world")

'{"all_tokens": 4, "used_tokens": 4, "real_probability": 0.5563012361526489, "fake_probability": 0.4436987340450287}'